## Prepare Dataset for Machine Learning


### Load Data

In [ ]:
import pandas as pd

df = pd.read_csv("data/breast-cancer-wisconsin.data", header=None) #file contains no header info
print(f"Read in {len(df)} rows")
df.head()

Read in 699 rows


,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


### Deal with Missing values

There are 16 missing attribute values and we need to deal with them as they have the value '?' in them. We need integer values for processing the data. So let's deal with the missing values.

In [ ]:
df.replace("?", 10000, inplace=True) #10,000 is way beyond the range of columns provided so acts as an outlier

### Feature Selection

The first column in the dataset is defined to be a "sample code number". This column should not have any bearing on the outcome of the test. So we will drop this column from our dataframe.

In [ ]:
df.drop([0], 1, inplace=True)
df.head()

,1,2,3,4,5,6,7,8,9,10
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2


### Split dataset into Train and Test

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X = np.array(df.drop([10], 1)) #last column contains label, so ignore it when creating X
y = np.array(df[10]) #last column contains label which is our y

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=43)

## Train Models

We will train different models from the sklearn library on this data and see how each one performs.

In [ ]:
names = ["Logistic Regression", "Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [ ]:
classifiers = [
    LogisticRegression(max_iter=300),
    KNeighborsClassifier(),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5, random_state=43),
    RandomForestClassifier(max_depth=5, random_state=43),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

In [ ]:
# iterate over classifiers
for name, clf in zip(names, classifiers):
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print(f"Accuracy of {name} Classifier is:{score}")


Accuracy of Logistic Regression Classifier is:0.9714285714285714
Accuracy of Nearest Neighbors Classifier is:0.9542857142857143
Accuracy of Linear SVM Classifier is:0.96
Accuracy of RBF SVM Classifier is:0.84
Accuracy of Gaussian Process Classifier is:0.96
Accuracy of Decision Tree Classifier is:0.9142857142857143
Accuracy of Random Forest Classifier is:0.9771428571428571
Accuracy of Neural Net Classifier is:0.8857142857142857
Accuracy of AdaBoost Classifier is:0.9542857142857143
Accuracy of Naive Bayes Classifier is:0.96
Accuracy of QDA Classifier is:0.9542857142857143


Now lets see if we can improve the performance of these algorithms.

Standardize features by removing the mean and scaling to unit variance. Will be used with some of the algorithms.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(random_state=43, class_weight={4: 2}, max_iter=500, n_jobs=-1)
lr_model.fit(X_train, y_train)
lr_accuracy = lr_model.score(X_test, y_test)
print(f"Accuracy of Logistic Regression Classifier is:{lr_accuracy}")

Accuracy of Logistic Regression Classifier is:0.9885714285714285


We managed an improvement of over 1.7% in the overall accuracy score from 97.14% to 98.85%.

### K Nearest Neighbor Algorithm

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
knn_model.fit(X_train_scaled, y_train)
knn_accuracy = knn_model.score(X_test_scaled, y_test)
print(f"Accuracy of kNN Classifier is:{knn_accuracy}")

Accuracy of kNN Classifier is:0.9657142857142857


We managed an improvement of over 1.1% in the overall accuracy score from 95.42% to 96.57%.

### Linear Support Vector Machines (SVM)

In [ ]:
from sklearn import svm

svm_model = svm.SVC(random_state=43, kernel='linear', class_weight={4: 2})
svm_model.fit(X_train, y_train)

svm_accuracy = svm_model.score(X_test, y_test)
print(f"Accuracy of Linear SVM Classifier is:{svm_accuracy}")

Accuracy of Linear SVM Classifier is:0.9885714285714285


We managed an improvement of over 2.8% in the overall accuracy score from 96.00% to 98.85%.

### RBF Support Vector Machines (SVM)

In [ ]:
from sklearn import svm
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)
param_grid = dict(gamma=gamma_range, C=C_range)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=43)
grid = GridSearchCV(SVC(), param_grid=param_grid, cv=cv)
grid.fit(X_train_scaled, y_train)

print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 10000000.0, 'gamma': 1e-06} with a score of 0.95


In [ ]:
rbf_svm_model = svm.SVC(gamma=0.01, C=100, class_weight={4: 2})
rbf_svm_model.fit(X_train_scaled, y_train)

rbf_svm_accuracy = rbf_svm_model.score(X_test_scaled, y_test)
print(f"Accuracy of RBF SVM Classifier is:{rbf_svm_accuracy}")

Accuracy of RBF SVM Classifier is:0.9828571428571429


We managed an improvement of over 14.2% in the overall accuracy score from 84.00% to 98.28%.

### Gaussian Process Classifier

In [ ]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

kernel = 1.0 * RBF(1.0)
gpc_model = GaussianProcessClassifier(kernel, random_state=43, max_iter_predict=1000, n_jobs=-1)
gpc_model.fit(X_train_scaled, y_train)

gpc_accuracy = gpc_model.score(X_test_scaled, y_test)
print(f"Accuracy of Gaussian Process Classifier is:{gpc_accuracy}")

Accuracy of Gaussian Process Classifier is:0.9657142857142857


We managed an improvement of over 0.5% in the overall accuracy score from 96.00% to 96.57%.

### Decision Trees

In [ ]:
from sklearn import tree

max_depth_range = np.linspace(1, 10, 10)
param_grid = dict(max_depth=max_depth_range)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=43)
grid = GridSearchCV(DecisionTreeClassifier(class_weight={4: 2}), param_grid=param_grid, cv=cv)
grid.fit(X_train, y_train)

print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))



The best parameters are {'max_depth': 4.0} with a score of 0.94


In [ ]:
tree_model = tree.DecisionTreeClassifier(class_weight={4: 2}, max_depth=4, random_state=43)
tree_model.fit(X_train, y_train)

tree_accuracy = tree_model.score(X_test, y_test)
print(f"Accuracy of Decision Tree Classifier is:{tree_accuracy}")

Accuracy of Decision Tree Classifier is:0.96


We managed an improvement of over 4.58% in the overall accuracy score from 91.42% to 96.00%.

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

max_depth_range = np.linspace(1, 10, 10)
max_features_range = np.arange(1, 10, 1)
param_grid = dict(max_depth=max_depth_range, max_features=max_features_range)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=43)
grid = GridSearchCV(RandomForestClassifier(class_weight={4: 2}, random_state=43), param_grid=param_grid, cv=cv)
grid.fit(X_train, y_train)

print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'max_depth': 4.0, 'max_features': 2} with a score of 0.97


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(class_weight={4: 2}, max_depth=4, n_estimators=300, max_features=2, random_state=43, n_jobs=-1)
rf_model.fit(X_train, y_train)

rf_accuracy = rf_model.score(X_test, y_test)
print(f"Accuracy of Random Forest Classifier is:{rf_accuracy}")

Accuracy of Random Forest Classifier is:0.9771428571428571


We are unable to improve score from 97.71%.

### Neural Network

In [ ]:
from sklearn.neural_network import MLPClassifier

nn_model = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100,), random_state=43, max_iter=1000, learning_rate='adaptive')
nn_model.fit(X_train_scaled, y_train)
nn_accuracy = nn_model.score(X_test_scaled, y_test)
print(f"Accuracy of MLP Classifier is:{nn_accuracy}")

Accuracy of MLP Classifier is:0.9428571428571428


We managed an improvement of over 5.85% in the overall accuracy score from 88.57% to 94.42%.

### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada_model = AdaBoostClassifier(random_state=43, n_estimators=100)
ada_model.fit(X_train, y_train)
ada_accuracy = ada_model.score(X_test, y_test)
print(f"Accuracy of Ada Boost Classifier is:{ada_accuracy}")

Accuracy of Ada Boost Classifier is:0.96


We managed an improvement of over 0.58% in the overall accuracy score from 95.42% to 96.00%.

### Gaussian Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb_model = GaussianNB()
gnb_model.fit(X_train, y_train)
gnb_accuracy = gnb_model.score(X_test, y_test)
print(f"Accuracy of Gaussian Naive Bayes Classifier is:{gnb_accuracy}")

Accuracy of Gaussian Naive Bayes Classifier is:0.96


We are unable to improve score from 96.00%.

### Quadratic Discriminant Analysis

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

qda_model = QuadraticDiscriminantAnalysis()
qda_model.fit(X_train, y_train)
qda_accuracy = qda_model.score(X_test, y_test)
print(f"Accuracy of Quadratic Discriminant Analysis Classifier is:{qda_accuracy}")

Accuracy of Quadratic Discriminant Analysis Classifier is:0.9542857142857143


We are unable to improve score from 95.42%.